#### Once we have embedded all content to vectors, we can save them to vector database

Here we use pgvector from Amazon RDS

In [105]:
!pip install -U psycopg2-binary pgvector tqdm 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [106]:
import psycopg2
from pgvector.psycopg2 import register_vector
import boto3 
import json 

client = boto3.client('secretsmanager')

response = client.get_secret_value(
    SecretId='rdspg-vector-secret'
)
database_secrets = json.loads(response['SecretString'])

In [107]:
dbhost = database_secrets['host']
dbport = database_secrets['port']
dbuser = database_secrets['username']
dbpass = database_secrets['password']

dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
dbconn.set_session(autocommit=True)

cur = dbconn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
register_vector(dbconn)
cur.execute("DROP TABLE IF EXISTS pdfembedding;")

In [108]:
cur.execute("""CREATE TABLE IF NOT EXISTS pdfembedding(
               id bigserial primary key, 
               content text, 
               pdf_file_name text, 
               page_number int, 
               pdf_file_path text, 
               content_embeddings vector(1024));""")

In [109]:
data_file_name = "./data/embedding.json"

In [110]:
with open(data_file_name, 'r') as f:
    data = json.load(f)

#### Check the embedding size equals to 1024

In [111]:
print(len(data[0]['embedding']) == 1024)

True


In [112]:
for item in data:
    cur.execute("""INSERT INTO pdfembedding
                      (content, pdf_file_name, page_number, pdf_file_path, content_embeddings) 
                  VALUES(%s, %s, %s, %s, %s);""", 
                  (item.get('content'), item.get('pdf_file_name'), item.get('page_number'), item.get('pdf_file_path'), item.get('embedding') ))

In [113]:
cur.execute("""CREATE INDEX ON pdfembedding 
               USING ivfflat (content_embeddings vector_l2_ops) WITH (lists = 100);""")

In [114]:
cur.execute("VACUUM ANALYZE pdfembedding;")

In [115]:
cur.execute("""SELECT id, content, pdf_file_name, page_number, pdf_file_path, content_embeddings 
            FROM pdfembedding 
            limit 3;""",
           )
r = cur.fetchall()
r

[(1,
  '2022 ANNUAL REPORT\n2022 ANNUAL REPORT',
  'annual_report.pdf',
  0,
  './pdfs/annual_report.pdf',
  array([-0.01685877, -0.02806014,  0.02163149, ...,  0.02593217,
          0.00515355,  0.03745385], dtype=float32)),
 (2,
  'This annual report (in both English and Chinese versions) \nhas been posted on the Company’s website at www.mi.com and the Stock Exchange’s website at www.hkexnews.hk. Shareholders who have chosen to rely on copies of the corporate communications (including but not limited to annual report and \n(where applicable) summary financial report, interim report and \n(where applicable) summary interim report, notice of meeting, listing document, circular and proxy form) posted on the aforesaid websites in lieu of any or all the printed copies thereof may request the printed copy of the annual report.\nShareholders who have chosen or are deemed to have consented \nto receive the corporate communications using electronic means and who have difficulty in receiving o

In [116]:
cur.close()
dbconn.close()
print ("Vector embeddings has been successfully loaded into PostgreSQL")
     

Vector embeddings has been successfully loaded into PostgreSQL


## Evaluate PostgreSQL vector Search Results

In this step we will use SageMaker realtime inference to generate embeddings for the query and use the embeddings to search the PostgreSQL to retrive the nearest neighbours and retrive the relevent content.

In [117]:
dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
dbconn.set_session(autocommit=True)
cur = dbconn.cursor()

#### The embedding model used for knowledge base (pdf) embedding should be the same for query embedding

In [118]:
endpoint_name = "jumpstart-dft-mx-tcembedding-robertafin-large-uncased-2"

In [119]:
query = """
99\n2022 ANNUAL REPORT\nShareholders’ Rights\nTo safeguard the Shareholders’ interests and rights, separate resolutions are proposed at the Shareholders’ meetings on \neach substantial issue, including the election of individual directors, for the Shareholders’ consideration and voting. All resolutions put forward at the Shareholders’ meetings will be voted by poll pursuant to the Listing Rules and poll results will be posted on the websites of the Company (www.mi.com) and the Stock Exchange after each Shareholders’ meeting.\nPursuant to the Articles of the Company, extraordinary general meetings shall be convened on the written requisition \nof any one or more Shareholders holding, as of the date of deposit of the requisition, in aggregate shares representing not less than one-tenth of the paid up capital of the Company which carry the right of voting at general meetings of the Company. A written requisition shall be deposited at the principal office of the Company in Hong Kong to the Board or the joint company secretaries for the purpose of requiring an extraordinary general meeting to be called by the Board for the transaction of any business specified in such requisition. Such meeting shall be held within two months after the deposit of such requisition. If within 21 days of such deposit, the Board fails to proceed to convene such meeting, the requisitionist(s) themselves may convene the general meeting in the same manner, and all reasonable expenses \nincurred by the requisitionist(s) as a result of the failure of the Board shall be reimbursed to them by the Company.\nFor the avoidance of doubt, Shareholders must deposit and send the original duly signed written requisition, notice \nor statement (as the case may be) to the Company’s principal place of business in Hong Kong and provide their full name, contact details and identification in order to give effect thereto. Shareholders’ information may be disclosed as required by law.\nThere is no provision allowing the Shareholders to move new resolutions at general meetings under the Cayman \nIslands Companies Law or the Articles. Shareholders who wish to move a resolution may request the Company to \nconvene a general meeting following the procedures set out in the preceding paragraph.\nWith respect to the Shareholders’ right in proposing persons for election as Directors, please refer to the procedures \navailable on the website of the Company.\nDuring the Reporting Period, the Company has amended its Memorandum and Articles of Association by way of a \nspecial resolution passed on June 2, 2022. Details of the amendments are set out in the circular dated April 27, 2022 to the Shareholders.\nThe up-to-date version of the Memorandum and Articles of Association is available on the websites of the Company \nand the Stock Exchange.\nEvents after the Reporting Period\nSave as disclosed in this Corporate Governance Report, there has been no other significant event subsequent to the Reporting Period and up to the Latest Practicable Date that might affect the Group. 
"""

In [120]:
from sklearn.preprocessing import normalize
import numpy as np 

def chunk_words(sequence, chunk_size):
    sequence = sequence.split()
    return [' '.join(sequence[i:i+chunk_size]) for i in range(0, len(sequence), chunk_size)]

def query_endpoint(payload):
    embeddings = []
    client = boto3.client("sagemaker-runtime")
    chunk_payload = chunk_words(payload, 400)
    for i, chunk in enumerate(chunk_payload):
        #print("Chunk ",i)
        #print("Content ",chunk)
        response = client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/x-text",
            Body=json.dumps(chunk),
        )    
        response = response["Body"].read().decode("utf8")
        response = json.loads(response)
        embeddings_chunk = response["embedding"]
        embeddings.append(embeddings_chunk)
    return embeddings

def parse_response(query_response):
    """Parse response and return the embedding."""
    embeddings = np.array(query_endpoint(query_response))
    #avg_embeddings = np.mean(embeddings, axis=0)
    # try max pooling of embedding vector
    avg_embeddings = np.max(embeddings, axis=0)

    avg_embeddings = avg_embeddings.reshape(1, -1)
    # normalization before inner product
    avg_embeddings = normalize(avg_embeddings, axis=1, norm='l2')
    return np.squeeze(avg_embeddings)

In [121]:
query_response = parse_response(query)
print(len(query_response))

1024


In [122]:
query_vector = query_response

In [123]:
print(type(query_vector))
print(query_vector)

<class 'numpy.ndarray'>
[ 0.00119371  0.02671994  0.01099941 ...  0.02109275 -0.01836102
  0.05742953]


#### Let's extract the top 3 most relevant pages from our database

In [124]:
# Execute the query
cur.execute("""
    SELECT id, content, pdf_file_name, page_number, pdf_file_path, content_embeddings
    FROM pdfembedding
    ORDER BY content_embeddings <=> %s limit 5;
    """, 
    (query_vector,)
)

In [125]:
# Fetch the results
results = cur.fetchall()

In [126]:
results

[(101,
  '99\n2022 ANNUAL REPORT\nShareholders’ Rights\nTo safeguard the Shareholders’ interests and rights, separate resolutions are proposed at the Shareholders’ meetings on \neach substantial issue, including the election of individual directors, for the Shareholders’ consideration and voting. All resolutions put forward at the Shareholders’ meetings will be voted by poll pursuant to the Listing Rules and poll results will be posted on the websites of the Company (www.mi.com) and the Stock Exchange after each Shareholders’ meeting.\nPursuant to the Articles of the Company, extraordinary general meetings shall be convened on the written requisition \nof any one or more Shareholders holding, as of the date of deposit of the requisition, in aggregate shares representing not less than one-tenth of the paid up capital of the Company which carry the right of voting at general meetings of the Company. A written requisition shall be deposited at the principal office of the Company in Hong K

In [127]:
cur.close()
dbconn.close()